<img align="left" src="https://ithaka-labs.s3.amazonaws.com/static-files/images/tdm/tdmdocs/tapi-logo-small.png" />

This notebook free for educational reuse under [Creative Commons CC BY License](https://creativecommons.org/licenses/by/4.0/).

Created by J.D. Porter  for the 2024 Text Analysis Pedagogy Institute, with support from [Constellate](https://constellate.org).
<br />
____

# `Small Language Models` `3`

This is lesson `3` of 3 in the educational series on `small language models`. This notebook is intended `to teach a basic strategy for building a word vector model from a small corpus of texts`. 

**Skills:** 
* Text analysis
* Python

**Audience:** `Teachers` / `Learners` / `Researchers`

**Use case:** `Tutorial`

**Difficulty:** `Beginner`

**Completion time:** `90 minutes`

**Knowledge Required:** 
```
* Python basics (variables, functions, lists, dictionaries)
```

**Knowledge Recommended:**
```
n/a
```

**Learning Objectives:**
After this lesson, learners will be able to:
```
1. Turn collocate data about a text into an array
2. Implement various distance measures for vectors
3. Play with parameters for a vector model
4. Find the most similar words to some target word
```
**Research Pipeline:**
```
If you want to try some of this on your own, you can:
1. Gather some texts and save them on your machine as .txt files
2. Use whatever steps you're interested in from this notebook
3. Interpret!
```
___

# Required Python Libraries
 * To keep things simple, we will try to work with very few libraries in this notebook (just 'os', which you do not need to install, though you do need to import it)

In [ ]:
### Import Libraries ###
import os
from scipy import spatial

# Required Data

**Data Format:** 
* plain text (.txt)

**Data Source:**
* included files (though you may supplement these whenever you feel comfortable doing so!)

**Data Quality/Bias:**

Included texts are from freely available sources like Project Gutenberg and Wikipedia. They have not been vetted for textual accuracy relative to, say, a scholarly edition.

**Data Description:**

This lesson uses textual data in .txt format (utf-8 encoding) from various sources.

# Introduction: The idea of distance

## Two kinds of distance

In session 2 on Wednesday, we talked about the idea of words appearing near each other. That's one way to think about proximity. When we encounter a phrase in a text, like "he is considered as the rightful property of some one or other of their daughters", we can say that words like "other" and "daughters" are *near* each other in that phrase. Or think of our example of the words "rightful" (which only appears in the second sentence of *Pride and Prejudice*) and "uniting" (which doesn't appear until the last sentence). I think it's pretty intuitive to see those words as being "far apart"—and when it comes to KWICs, they are!

In other text mining situations, though, you'll see the idea of "distance" used quite differently. Let's think back to the example from the session 1, on Monday. Here's the two dimensional graph of our bird terms:

     |
     |
     |  penguin
     |
    "|
    s|                              duck
    w|
    i|
    m|
    s|
    "|
     |
     |
     |                                               eagle
     | 
     ------------------------------------------------------>
                        
                        "flies"

If we want to measure the distance between "duck" and "eagle" on this graph, we can just draw a line from one to the other and see how long it is. That's a perfectly valid way to see how "close" they are to each other, and in fact it's what we mean when we talk about "distance" in the context of word vectors. But notice that this graph doesn't tell us anything about whether the words appear near each other *in a text*. We don't know if our corpus ever says "eagle" within 100 pages of "duck", or even in the same document. All we know is that the words are in similar ways relative to "flies" and "swims".

So we have two forms of distance here:
 1. Appearing next to each other "on the page" (we don't always literally have pages, but you get the idea) 
 2. The relative positions of words based on their arrangement in a lexical *space*

As you might have noticed, the second kind of distance depends on the first. After all, our x axis just shows how often our bird terms appear *near* the word "flies" in the first sense of the term—i.e., we're seeing lots of phrases that say things like "the eagle flies over the lake". The same goes for the y axis and the word "swims". 

You might wonder: What's the point of this "meta" distance? Can't we just measure the first kind? The short answer is yes. That's exactly what we do when we look at collocates. The collocates of "duck" and "eagle" would include "flies". Yet the second kind of distance tells us things that the first kind can't. When we arrange our bird words in space like this, we see *how the words are used*. Eagles may never be used *near* ducks (or used near them only rarely), but we talk about a lot of similar things when we talk about either. 

We think of similarity in these terms all the time in real life. It's entirely plausible that you've never seen the names Emily Dickinson and Amanda Gorman in the same sentence before, but the two are obviously similar—they come up near words like "poet" and "writer" all the time. Two socialist 76ers fans who love pho, bluegrass, and *The Bachelorette* might be similar even if they live on opposite coasts, never appearing "near" each other in an immediate sense.

You might notice that I've started talking about "similarity" rather than "distance". This is a common move when we're discussing that second kind of "meta" distance. I think it tracks a subtle shift in meaning compared to the first kind of distance. Though we can measure the length of a line between "eagle" and "duck" above in repeatable, quantifiable terms, we're not really in "space" in the ordinary sense when we do it. There is no background space in a word vector model. The dimensions are just other words; there is nothing "outside" the words anchoring them in place. (This has the trippy side effect that all potential points are also potential dimensions, but we don't need to go too far down that rabbit hole for now.) So what we're really seeing is how *similarly* the words are used in our texts.

With that theoretical apparatus in place, we can turn to the "repeatable, quantifiable" part. Let's accept that the second kind of distance is meaningful. How do we measure it?

## Measuring distance

The easiest way to think about distance is to start with a version that most of us learned in junior high or high school. Here's the data for our bird words again:

| term | "flies" | "swims" | "hockey" |
| ---- | ---- | ---- | ---- |
|penguin | 1 | 9 | 9
|duck | 8 | 7 | 8
|eagle | 10 | 1 | 0

Let's measure the distance between "duck" and "eagle" on the two dimensional graph above. Eagle has a 10 for "flies", and duck has an 8, so they're 2 apart on the x axis. On the y axis, duck is 7 and eagle is 1, so they're 6 apart. You may have noticed that we've just drawn a right triangle, which means we can calculate a hypotenuse with the help of Pythagoras:

a<sup>2</sup> + b<sup>2</sup> = c<sup>2</sup>

or

2<sup>2</sup> + 6<sup>2</sup> = 40

so

c = the square root of 40, or roughly 6.3.

This is a nice, intuitive way to measure distance. If we scroll back up the graph above and draw a straight line from duck to eagle, it will be 6.3 units long. (Or anyway, it would be if I had drawn that graph more precisely.) However, it's not the only way to measure distance. We could, for instance, think in terms of Manhattan distance (also known as city block distance). 

Let's say I was near Union Square in New York City, at 15th Street and 5th Avenue. I'm meeting a friend in Chelsea, at 27th and 10th. In that part of town, the streets are a grid. In Euclidean terms, the distance I need to walk is:

(27-15)<sup>2</sup> + (5-10)<sup>2</sup> = c<sup>2</sup>

so c = 13.

However, this measurement is useless to me, because I can't walk straight from my corner to my friend's corner—there are a bunch of buildings and stuff in the way. I can only walk on the streets, making right angle turns. That means the *real* distance I'm going to walk is given by 

(27-15) + (5-10)

so it's 17 (five blocks west plus 12 blocks north).

I mention this form of distance mainly because I think it's a pretty straightforward example of another way to think about things. We're just measuring along the dimensions that we've got, our streets and avenues. 

In text mining, we often use cosines when calculating distance or similarity. This is tough to draw in markdown, but if you draw a line from the origin to "duck", and another line to "eagle", you can then measure the cosine of the angle between them. The bigger the angle, the farther apart the two words are (notice above that the angle between "penguin" and "eagle" would be much larger than the angle between "duck" and "eagle"). The advantage of cosine distance over other metrics is that it doesn't vary as dramatically with absolute values. People don't say "ptarmigan" as *often* as they say "pigeon", but their relationship to other words is similar—that is, we'd expect both to be used near "feathers" and "eggs" much more often than they're near "fins" or "astronauts".[$^{1}$](#1) 

There's a small wrinkle with cosine. Cosines decrease as angles increase, so we actually expect a *small* cosine with a *big* angle, i.e. the cosine goes down as the difference between two terms grows larger. So we call the measure *cosine similarity* (the higher it is, the more similar the two terms are). We use 1-cosine to reflect *cosine distance*. The two measures capture exactly the same information, but the latter is more intuitive in comparison with other forms of distance like Euclidean or Manhattan.[$^{2}$](#2)

One final issue before we move on to coding: What happens to these calculations when we add dimensions? Fortunately, the answer is: not much! This is easiest to see with Manhattan distance. If our friend is also on the 30th floor of a building (I think the address I gave in New York is actually a park in real life, but let's pretend), all we'd do to get the total distance is add the distance up in the air—the z axis. The same holds for Euclidean distance (if we add dimension z, **c<sup>2</sup>** would equal **a<sup>2</sup>** + **b<sup>2</sup>** + **z<sup>2</sup>**). Cosine distance works in much the same way—and since we'll just be using a Python library to calculate it, the extra dimensions won't pose any problem at all!

<hr></hr>

**Footnotes**

1. <a id="1"></a> For more on cosine vs Euclidean distance, see footnote 11 on page 252 of this paper (also a great resource on the distributional hypothesis in general): Baroni, Marco, Raffaella Bernardi, and Roberto Zamparelli. "Frege in space: A program for compositional distributional semantics." *Linguistic Issues in language technology* 9 (2014): 241-346.
2. <a id="2"></a> For a nice primer on calculating cosine similarity/distance by hand, [see this site](https://www.geeksforgeeks.org/cosine-similarity/).

# Practicing measuring distance

We'll be using the Scipy library to do our distance calculations. Let's see it in action for a few simple examples. 

Imagine we have two points on a graph. We can depict them as lists:

```Python
x = [2,3]
y = [5,7]
```

That might look something like this:

     |
     |
     |
     |             y  
     |
     |
     |    x
     |
     |
     |__ __ __ __ __ __ __ __ __ __

The horizontal distance is 3, and the vertical distance is 4.

In [ ]:
# Loading our points
x = [2,3]
y = [5,7]

# What if y was a much more common word with a similar use pattern?

# What if we had three dimensions?
# x = [2,3,2]
# y = [5,7,14]

# Gathering/arranging data from our corpus

Before we calculate any distances, we're going to need some data. Let's take a detour back to material from earlier this week, and figure out the collocates of words in our jazz corpus.

In [ ]:
# Here are a few useful functions we've built together

# Takes a directory and returns a list of files of the specified type
def dir2files(somedir,file_type='.txt',path=True):
    files = os.listdir(somedir)
    if path:
        files = [os.path.join(somedir,f) for f in files]
    files = [f for f in files if f.endswith(file_type)]
    return files

# Takes a word and returns a cleaned up version of it
# This works a little differently than the version from Monday
def cleanword(someword,remove_apostrophe=True):
    word = someword.casefold()
    while word and not word[0].isalpha():
        word = word[1:]
    while word and not word[-1].isalpha():
        word = word[:-1]
    return word

# Takes a txt filename and returns a list of its words
def file2words(somefilename,clean = True):
    with open(somefilename) as source:
        text = source.read()
    words = text.split()
    if clean:
        words = [cleanword(w) for w in words]
    return words

# Takes a count dictionary and returns a sorted list of its key,value pairs
def sortdict(somecountdict,reverse_direction=True):
    s = sorted(somecountdict,key = lambda i:somecountdict[i],reverse=reverse_direction)
    sorted_counts = [(word,somecountdict[word]) for word in s]
    return sorted_counts

# Takes a target term and find its KWICs in some list of words
def get_KWIC(sometargetterm,somewords,window = 10,excl_target = True):
    KWIC = []
    for n,word in enumerate(somewords):
        if word == sometargetterm:
            start = max(n-window,0)
            end = min(n+window+1,len(somewords)-1)
            if excl_target:
                k = somewords[start:n] + somewords[n+1:end]
            else:
                k = somewords[start:end]
            KWIC.append(k)
    return KWIC

# Take a list of KWICs and return a dictionary of word counts
def kwic2coll(somekwiclist):
    counts = {}
    for k in somekwiclist:
        for w in k:
            if w not in counts:
                counts[w] = 0
            counts[w] += 1
    return counts

In [ ]:
# Let's begin by counting all of the words in our corpus


In [ ]:
# Grabbing all of the words in order


In [ ]:
# Looking at a few words


## Dimension reduction

As we've seen, building out our array to show how every word relates to every single other word leaves us with a table that is both unwieldy and sparse (a term of art for "filled with zeroes"). We can solve both problems at once by making a smaller table, trading off some comprehensiveness to gain some efficiency. Since we're basically chopping dimensions off of our table, we can call this process "dimension reduction".

This is where many word vector models take a leap in complexity. You can imagine many ways to try to maximize the information from the dimensions we keep, while minimizing the number we need. Starting small, we could *lemmatize* our data. Perhaps "am", "is", "are", "was", and "were" could all be reduced to "be". Or we could look for words that seem to correlate with each other and group them together—perhaps using the collocate data that we're collecting anyway. Or we can group words based on the way they influence the relative probabilities of the presence of other words in a series of windows across a text (a rough approximation of what Google does in the Continuous Bag of Words archictecture for word2vec).

We're not going to do any of that. We want a simple, interpretable, computationally tractable model that can tell us things about our words, so we're going to approach this with two very simple assumptions:

1. The most common words do the most work in a corpus
2. Stop words aren't very semantically informative

In [ ]:
# Decide how many dimensions we want

# Get a list of stopwords

# Get the right number of common words that aren't stopwords


# Making the model

## Building the array

In [ ]:
# Optionally, filter words

# Optionally, filter stops

# Gather the data

# Build the array


## Exploring the results

In [ ]:
# For any given word, gather its distance to every other word


### A note on window size

Here's an interesting passage from the Baroni, Bernardi, and Zamparelli piece cited in footnote 1, above:

"Different contexts capture different kinds of semantic similarity or “relatedness” (Budanitsky and Hirst 2006). At the two extremes, counting documents as contexts captures “topical” relations (the words *war* and *Afghanistan* will have a high cosine, because they often co-occur in documents), whereas DSMs based on word co-occurrence within narrow windows or constrained by syntactic relations tend to capture tighter “taxonomic” relations (such as the one between *dog* and *hyena*). Unsurprisingly, no single definition of context is appropriate for all tasks, and the jury on the “best” context model is still out (Sahlgren 2008)." (251)

### Why 'verification'?

With some window sizes, the word "verification" appears to be highly similar to our target terms, even when this seems at odds with reality (is "verification" much like "swing"? is a question I don't really know how to parse). This happens for a few other words sometimes, too, such as "citations". This is an oddity of our method. The word "verification appears in our corpus exclusively in Wikipedia boilerplate that looks like this:

    This section needs additional citations for verification. Please help improve this article by adding citations to reliable sources in this section. Unsourced material may be challenged and removed. Find sources: "Paul Motian" – news · newspapers · books · scholar · JSTOR (March 2024) (Learn how and when to remove this message)

If the window size is small enough, "verification" will never appear near any of the words that form the dimensions of our array. As a result, its vector will be a series of 0's—it will be located at the origin of our many-dimensional space. This causes problems for spatial's distance calculation, and results in a value of 0.

How should we deal with this? Clean up our corpus? Change our dimension reduction approach? Filter our results to remove these oddballs? There are many defensible options depending on our goals and our willingness to add caveats to our final output. In the meantime, this is a nice reminder of the limitations of our small model!